In [ ]:
#製作環境
from flask import Flask, request, jsonify
import pymysql
import datetime

In [ ]:
#呼叫出Flask
app = Flask(__name__)

In [ ]:
#建立與mysql的連線
conn = pymysql.connect(host='db', port=3306, user='root', passwd='iii', db='chatbot_db',charset='utf8mb4')


In [ ]:
#方便用來跟mysql互動
cur = conn.cursor()

In [ ]:
#接口功能：檢視所有使用者資訊
#接口位置：/users，使用get的http method
@app.route('/users',methods=['GET'])
def read_users():
    #找出資料庫內的所有user資料
    cur.execute('SELECT * FROM chatbot_db.users')
    #由於是多筆，使用fetchall
    user = cur.fetchall()

    #裝成矩陣格式
    answer = []
    for i in user:
        result = {
                "user_open_id":i[0],
                "user_nick_name":i[1],
                "user_status" : i[2],
                "user_img" : i[3],
                "user_register_date" : i[4],
                "user_register_menu" : i[5]
            }
        answer.append(result)
      
    #轉成json格式
    return jsonify(answer)

In [ ]:
#接口功能：對資料庫新增使用者資料
#接口位置：/users，使用post的http method
@app.route('/users',methods=['POST'])
def add_user():
    #抓跑function時的時間
    time = datetime.datetime.utcnow().strftime("%Y%m%d%H%M")
    #將傳過來的json檔擷取出來
    a = request.get_json()

    #插入資料庫中，要按照資料庫的格式塞入
    insertsql=("INSERT INTO chatbot_db.users VALUES ( %s,%s,%s,%s,%s,%s )") 
    value = (a['user_open_id'],
                 a['user_nick_name'],
                 a['user_status'],
                 a['user_img'],
                 time,
                 a['user_register_menu'])
    cur.execute(insertsql , value)
    #將資料送進資料庫中
    conn.commit()
    #回傳一個正確的描述
    result =  { "status_describe":"success add user" }
    return jsonify(result)

In [ ]:
#接口功能：更新指定使用者資訊
#接口位置：/users/<userid>，使用put的http method
@app.route('/users/<userid>',methods=['PUT'])
#由於使用url parameter 所以有擷取userid
def update_user(userid):
    #取出傳過來的資料
    a = request.get_json()
    #方便錯誤排除
    error = None
    
    #看看資料庫內是否有重複的id
    cur.execute('SELECT user_open_id FROM chatbot_db.users WHERE user_open_id = ("%s")' % (userid))
    test = cur.fetchone()
    #沒有重複的話
    if not test:
        error = 'Please enter the right id.'

    #沒有錯誤的話，將資料更新進資料庫
    if error is None:
        #更新資料庫
        insertsql=('UPDATE chatbot_db.users SET user_nick_name=("%s"),user_status=("%s"),user_img=("%s"),user_register_menu=("%s") WHERE user_open_id=("%s")') % (a['user_nick_name'],a['user_status'],a['user_img'],a['user_register_menu'],a["user_open_id"])
        cur.execute(insertsql)
        conn.commit()
        #回傳一個正確的描述
        result =  { "status_describe":"success update user" }
        return jsonify(result)
    
    
    #製作一個錯誤的描述
    result = {"status_describe":"{}".format(error)}
    
    #回傳轉檔成json格式的錯誤描述
    return jsonify(result)

In [ ]:
#__name__ == __main__ 代表你執行這個模塊（py檔）時會成立
#假如你是被別的檔案import的話，__name__ == 檔案名稱，這個if就不會成立
if __name__=='__main__':

    #運行flask server，運行在0.0.0.0:5000
    #要特別注意假如運行在127.0.0.1的話，會變成只有本機連的到，外網無法
    app.run(host='0.0.0.0',port=5000)